# 🔥 **자연어처리 필수 루틴!**

1. 말뭉치 준비
2. 토큰 분절 (tokenization)
3. 수치화 (numericalization)
4. 배치 (batch) 구성

### 1. 말뭉치 준비

In [ ]:
import pathlib
from Korpora import Korpora

this_dir = pathlib.Path().parent.resolve()
corpus = Korpora.load("nsmc", root_dir=f"{this_dir}/data").get_all_texts()

In [ ]:
# 어떻게 생겼는지 10개만 볼까요?
print(corpus[0:10])

### 2. 토큰 분절 (Tokenization)

토큰 분절 방식은 여러 가지가 있습니다.   
단어 또는 그 이상의 큰 단위부터, 자모 수준의 작은 단위도 가능합니다.   
현재 가장 많이 사용되는 토큰 분절 방법은 subword 단위의 알고리즘들입니다.

In [ ]:
# 먼저, SpaCy tokenizer부터 살펴봅시다.
!python -m spacy download ko_core_news_sm

In [ ]:
from torchtext.data import Field

processor = Field(
	sequential=True, use_vocab=True, init_token="<bos>", eos_token="<eos>",
	tokenize="spacy", tokenizer_language="ko_core_news_sm",
	pad_token="<pad>", unk_token="<unk>", batch_first=True,
)

In [ ]:
from torchtext.data import Dataset
from torchtext.data import Example
from tqdm import tqdm


class TokenizedDataset(Dataset):

    def __init__(self, corpus, processor):
        fields = [("text", processor)]
        examples = []
        for movie_comment in tqdm(corpus, desc="토큰 분절 중입니다"):
            each_example = [movie_comment]
            examples.append(
                # ``Example`` 객체가 생성되는 과정에서 토큰 분절이 일어나게 됩니다.
                Example.fromlist(each_example, fields)
            )
        super().__init__(examples, fields)

dataset = TokenizedDataset(corpus, processor)

In [ ]:
# 토큰 분절이 잘 되었는지 볼까요?
print([dataset[i].text for i in range(10)])

#### 👨‍💻 Byte-Pair Encoding 방식의 토큰 분절

In [ ]:
from sentencepiece import SentencePieceProcessor
from sentencepiece import SentencePieceTrainer

SentencePieceTrainer.train(input=f"{this_dir}/data/nsmc/ratings_train.txt", model_prefix="spm",
						   vocab_size=10000)
tokenizer = SentencePieceProcessor(model_file="./spm.model")

In [ ]:
encoded = tokenizer.encode("영화가 잘 안되도 좋습니다. 하지만 엄복동 하나만 기억해주세요", out_type=str)
print(encoded)

In [ ]:
bpe_func = lambda raw_text: tokenizer.encode(' '.join(raw_text), out_type=str)
processor = Field(
	sequential=True, use_vocab=True, init_token="<bos>", eos_token="<eos>",
    preprocessing=bpe_func,
	pad_token="<pad>", unk_token="<unk>", batch_first=True,
)

In [ ]:
dataset = TokenizedDataset(corpus, processor)
print([dataset[i].text for i in range(10)])

### 3. 수치화

이후 어떤 처리를 하든, 문자열보다는 숫자를 처리하는 것이 낫습니다.   
각 토큰을 어떻게 숫자로 바꿀까요?   
간단히, 모든 고유한 토큰의 개수가 $n$이라면, $0$부터 $(n-1)$까지 번호를 매기면 될 것입니다.   
이렇게 토큰과 번호를 짝지어주는 것이 바로 '어휘'입니다.

In [ ]:
processor.build_vocab(dataset)
# 만들어진 어휘는 얼마나 클까요?
print(f"어휘에 등재된 토큰 개수: {len(processor.vocab)}개")

In [ ]:
# 인덱스 순으로 정렬된 토큰을 10개만 봅시다.
print(processor.vocab.itos[:20])

In [ ]:
# 이번에는 반대로, 토큰을 하나 정해서 그 인덱스를 확인해 봅시다.
token = "▁영화"
print(f"``{token}``의 인덱스: {processor.vocab.stoi[token]}")

### 4. 배치 구성

In [ ]:
from torchtext.data import Iterator

batch_producer = Iterator(dataset, batch_size=10,
						  repeat=True, shuffle=False, sort=False)
for i, batch in enumerate(batch_producer):
	# ``torchtext.data.Batch`` 객체가 생성되는 과정에서 수치화가 일어나게 됩니다.
	# 그 결과, ``batch.text``는 ``torch.LongTensor``가 됩니다.
	print(batch.text.tolist())
	break